In [1]:
!python3 -m pip install phylovega --user

In [2]:
!python3 -m pip install biopython==1.77 --user

In [3]:
!python3 -m pip install jupyter-require --user

In [4]:
import pandas as pd
import dendropy
import numpy as np

In [5]:
import Bio
import phylopandas as ph
import phylovega
import IPython
import os

In [6]:
path = "/scratch/gs69042/bahllab/"
combined_df = pd.DataFrame()
for file in os.listdir(path):
    if '.nwk' in file:
        tr1 = ph.read_newick(path+file)
    else:
        continue
    print(file)
    parents = tr1.parent[1:].apply(int).sort_values(ascending=False).unique()
    for parent1 in parents:
        parent = str(parent1)
        if parent:
            parentid = "|".join(tr1[tr1.parent == parent].id.sort_values().apply(lambda x: x if "|" in x else x.split('/')[2]))

            tr1.loc[tr1.parent == parent, 'parent'] = parentid
            tr1.loc[tr1.id == parent, 'id'] = parentid
    tr1.loc[tr1.type == 'leaf', 'id'] = tr1.loc[tr1.type == 'leaf', 'id'].apply(lambda x: x.split("/")[2])
    tr1['source'] = file
    print(tr1.head())
    if len(combined_df) > 1:
        combined_df = pd.concat([combined_df, tr1])
    else:
        combined_df = tr1
    print("-------------------------")

GISAID_combined.0801-0930.aligned.nwk
   type                                                 id  \
0  root  TX-CDC-ASC210077066|TX-CDC-ASC210237810|TX-CDC...   
1  leaf                                TX-CDC-ASC210322426   
2  node  TX-CDC-ASC210077066|TX-CDC-ASC210237810|TX-CDC...   
3  node            TX-CDC-ASC210200572|TX-CDC-ASC210404253   
4  leaf                                TX-CDC-ASC210404253   

                                              parent   length  \
0                                               None  0.00000   
1  TX-CDC-ASC210077066|TX-CDC-ASC210237810|TX-CDC...  0.00055   
2  TX-CDC-ASC210077066|TX-CDC-ASC210237810|TX-CDC...  0.00055   
3  TX-CDC-ASC210077066|TX-CDC-ASC210237810|TX-CDC...  0.00055   
4            TX-CDC-ASC210200572|TX-CDC-ASC210404253  0.00055   

                                  label  distance         uid  \
0                                     0   0.00000  VbIBhedaLb   
1  hCoV-19/USA/TX-CDC-ASC210322426/2021   0.00055  WPJSUtZbhO   
2  

In [9]:
combined_df.to_csv('neo_ready.csv', index=False)

In [8]:
def tree_to_csv(tree, tree_id, tree_date, location):
    '''
    Purpose: take a tree, which was read from a nwk file format, 
             parse the relationships therein, and output a CSV 
             ready-made for Neo4j import.
    Variables: tree=dendropy tree object to interact with, tree_id=an 
             identifier for the tree objects, tree_date=date tree was 
             generated in YYYY/MM/DD format, location=location in file 
             system to drop the output csv. If an empty string, don't 
             output to file. 
    Returns: dataframe of relationship data, as it would be output in a CSV.
    Date authored: 11/29/2021
    Author: Garrick'''
    
    return df